In [1]:
import cv2
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import math
from sklearn.linear_model import LinearRegression
sns.set()
# sns.set(rc={'axes.facecolor':'white', 'figure.facecolor':'#cccccc'})
%matplotlib auto
from PIL import Image
import pandas as pd

Using matplotlib backend: Qt5Agg


In [2]:
df = pd.read_excel('cvals.xlsx')

In [3]:
df.head()

,ill,vp,wb,sq,r,g,b
0,1,1,0.0,w,186.0,188,216
1,1,1,NaN,m,189.0,95,128
2,1,1,NaN,c,18.0,132,173
3,1,1,NaN,y,206.0,195,136
4,1,2,NaN,w,178.0,183,217


In [46]:
# white points for each illuminant
# D * rawPicture = fixed picture
def getDByWhitePatch(df, ill):
    illDf = df[df.ill == ill ]
    illDf = illDf[illDf.sq == 'w']
    sumR = illDf.r.sum()
    sumG = illDf.g.sum()
    sumB = illDf.b.sum()
    R = sumR / illDf.shape[0]
    G = sumG / illDf.shape[0]
    B = sumB / illDf.shape[0]
    
    # we got the RGB in white. Fixed images have 255, 255, 255 for all the channels.
    D = np.zeros([3, 3])
    
    D[0, 0] = 255 / R
    D[1, 1] = 255 / G
    D[2, 2] = 255 / B
    
    return D

def getDByGrayWorld(df, ill):
    # gray world has rgb(128,128,128)
    illDf = df[df.ill == ill ]
    sumR = illDf.r.sum()
    sumG = illDf.g.sum()
    sumB = illDf.b.sum()
    R = sumR / illDf.shape[0]
    G = sumG / illDf.shape[0]
    B = sumB / illDf.shape[0]
    
    # we got the RGB in white. Fixed images have rgb(128,128,128) for all the channels.
    D = np.zeros([3, 3])
    
    D[0, 0] = 128 / R
    D[1, 1] = 128 / G
    D[2, 2] = 128 / B
    
    return D


def getMaxRGB(df):
    
    r, g, b = 0, 0 ,0 # holds values for maxBrightness color
    maxBrightness = 0
    
    for row in df.itertuples():
        brightness = row.r + row.g + row.b 
        if brightness > maxBrightness: 
            maxBrightness = brightness
            r = row.r
            g = row.g
            b = row.b
    
    return r, g, b
        
    
def getDByWhiteWorld(df, ill):
    # gray world has rgb(128,128,128)
    illDf = df[df.ill == ill ]
    
    R, G, B = getMaxRGB(illDf)
    
    print(f'R: {R}, G: {G}, B: {B}')
    
    # we got the RGB in white. Fixed images have rgb(128,128,128) for all the channels.
    D = np.zeros([3, 3])
    
    D[0, 0] = 255 / R
    D[1, 1] = 255 / G
    D[2, 2] = 255 / B
    
    return D
    
    

In [47]:
D1_WP = getDByWhitePatch(df, 1)
D2_WP = getDByWhitePatch(df, 2)
D3_WP = getDByWhitePatch(df, 3)

D1_GW = getDByGrayWorld(df, 1)
D2_GW = getDByGrayWorld(df, 2)
D3_GW = getDByGrayWorld(df, 3)

D1_WW = getDByWhiteWorld(df, 1)
D2_WW = getDByWhiteWorld(df, 2)
D3_WW = getDByWhiteWorld(df, 3)

R: 195.0, G: 196, B: 224
R: 179.0, G: 198, B: 238
R: 151.0, G: 204, B: 253


In [48]:
D1_WP

array([[1.37096774, 0.        , 0.        ],
       [0.        , 1.34351949, 0.        ],
       [0.        , 0.        , 1.15803815]])

In [49]:
D1_GW

array([[0.83344185, 0.        , 0.        ],
       [0.        , 0.7992507 , 0.        ],
       [0.        , 0.        , 0.73247496]])

In [50]:
D1_WW

array([[1.30769231, 0.        , 0.        ],
       [0.        , 1.30102041, 0.        ],
       [0.        , 0.        , 1.13839286]])

In [29]:
illDf

,ill,vp,wb,sq,r,g,b
0,1,1,0.0,w,186.0,188,216
4,1,2,NaN,w,178.0,183,217
8,1,3,NaN,w,195.0,196,224
12,1,4,NaN,w,184.0,189,221
16,1,5,NaN,w,187.0,193,223
